In [2]:
# Assignment 1

import numpy as np
import ast
import urllib
import matplotlib.pyplot as plt
import random
from collections import defaultdict
import scipy.optimize
from sklearn import linear_model
import math
from sklearn.ensemble import AdaBoostClassifier
import gzip

In [3]:
def parseDataFromFile(fname):
    """ Read and convert the input to a list of dicts"""
    for l in open(fname, encoding="utf-8"):
        yield ast.literal_eval(l)

In [4]:
def readGz(path):
    for l in gzip.open(path, 'rt', encoding="utf-8"):
        yield eval(l)

In [5]:
#fname = "C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\train.json"
dataset = list(readGz("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\train.json.gz"))
training_validation_boundary = 165000
training_dataset = dataset[:training_validation_boundary]
validation_dataset = dataset[training_validation_boundary:]

In [6]:
def perform_duplication(dataset, start, end):
    assert(start <= len(dataset) and end <= len(dataset))
    new_set = []
    print("Start = {}, end = {}".format(start, end))
    for d in dataset[start:end]:
        user, game = d['userID'], d['gameID']
        gamesNotPlayed = []
        for game in gameIDs:
            if(game not in gamesPerUser[user]):
                gamesNotPlayed.append(game)
                
        random.seed()
        randomGame = random.choice(gamesNotPlayed)
        new_data = {'hours': 0, 'gameID': str(randomGame),\
                'hours_transformed': 0, 'early_access': False,\
                'date': '', 'text': '', 'userID': str(userID)}
        new_set.append(new_data)
    return dataset + new_set

In [8]:
usersPerGame = defaultdict(set) # list of userIDs per game
gamesPerUser = defaultdict(set) # list of gameIDs per user
reviewsPerUser = defaultdict(list)
reviewsPerGame = defaultdict(list)
gameIDs = [] # This contains the entire list of games
hoursPerGame = defaultdict(list)
hoursPerUser = defaultdict(list)

usersPerGame_training = defaultdict(set) # list of userIDs per game
gamesPerUser_training = defaultdict(set) # list of gameIDs per user
reviewsPerUser_training = defaultdict(list)
reviewsPerGame_training = defaultdict(list)
gameIDs_training = [] # This contains the entire list of games
hoursPerGame_training = defaultdict(list)
hoursPerUser_training = defaultdict(list)

usersPerGame_validation = defaultdict(set) # list of userIDs per game
gamesPerUser_validation = defaultdict(set) # list of gameIDs per user
reviewsPerUser_validation = defaultdict(list)
reviewsPerGame_validation = defaultdict(list)
gameIDs_validation = [] # This contains the entire list of games
hoursPerGame_validation = defaultdict(list)
hoursPerUser_validation = defaultdict(list)

# Computing maps and lists for all samples
for d in dataset:
    userID, gameID, hours = d['userID'], d['gameID'], d['hours_transformed']
    usersPerGame[gameID].add(userID)
    gamesPerUser[userID].add(gameID)
    reviewsPerUser[userID].append(d)
    reviewsPerGame[gameID].append(d)
    hoursPerGame[gameID].append((userID, hours))
    hoursPerUser[userID].append((gameID, hours))
    if gameID not in gameIDs:
        gameIDs.append(gameID)
        
# Computing maps and lists for training set samples
for d in training_dataset:
    userID, gameID = d['userID'], d['gameID']
    usersPerGame_training[gameID].add(userID)
    gamesPerUser_training[userID].add(gameID) # list of gameIDs per user
    reviewsPerUser_training[userID].append(d)
    reviewsPerGame_training[gameID].append(d)
    hoursPerGame_training[gameID].append((userID, hours))
    hoursPerUser_training[userID].append((gameID, hours))
    if gameID not in gameIDs_training:
        gameIDs_training.append(gameID)
        
# Computing maps and lists for training set samples
for d in validation_dataset:
    userID, gameID = d['userID'], d['gameID']
    usersPerGame_validation[gameID].add(userID)
    gamesPerUser_validation[userID].add(gameID) # list of gameIDs per user
    reviewsPerUser_validation[userID].append(d)
    reviewsPerGame_validation[gameID].append(d)
    hoursPerGame_validation[gameID].append((userID, hours))
    hoursPerUser_validation[userID].append((gameID, hours))
    if gameID not in gameIDs_training:
        gameIDs_validation.append(gameID)

print("Data preparation phase successfully completed...")

Data preparation phase successfully completed...


In [9]:
new_training_set = perform_duplication(training_dataset, 0, len(training_dataset))
new_validation_set = perform_duplication(validation_dataset, 0, len(validation_dataset))
print("Lengths of new datasets = {} and {}".format(len(new_training_set), len(new_validation_set))) 

Start = 0, end = 165000
Start = 0, end = 10000
Lengths of new datasets = 330000 and 20000


In [10]:
# Compute the confusion matrix for the given set of predictions and actual values
def compute_confusion_matrix(Y_actual, Y_predict):
    """ This function is used to compute the confusion matrix"""
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    assert(len(Y_actual) == len(Y_predict))
    for i in range(len(Y_actual)):
        if(Y_actual[i] == 1 and Y_predict[i] == 1):
            TP += 1
        elif(Y_actual[i] == 0 and Y_predict[i] == 1):
            FP += 1
        elif(Y_actual[i] == 0 and Y_predict[i] == 0):
            TN += 1
        elif(Y_actual[i] == 1 and Y_predict[i] == 0):
            FN += 1
    return (TP, TN, FP, FN)

# Compute the ratios for computing the TP, TN, FP and FN
def compute_rates(TP, TN, FP, FN):
    TPR = TP / (TP + FN)
    TNR = TN / (TN + FP)
    FPR = FP / (FP + TN)
    FNR = FN / (FN + TP)
    BER = 0.5 * (FPR + FNR)
    return (TPR, TNR, FPR, FNR, BER)

In [11]:
def jaccard(s1, s2):
    numerator = len(s1.intersection(s2))
    denominator = len(s1.union(s2))
    return numerator/denominator

# Helps compute the cosine similarity
def cosine_similarity(s1, s2):
    """ This only works for binary values in the sets"""
    numerator = len(s1.intersection(s2))
    denominator = math.sqrt(len(s1) * len(s2))
    if(denominator == 0):
        return 0
    return numerator/denominator

In [14]:
def sim_pop(dataset, sim_th, pop_th):
    idx, numerator = 0, 0
    y_pred = []
    y_act = []
    for d in dataset:
        u, g = d['userID'], d['gameID']
        # Use hoursPerGame for popularity
        # User based comparison for similarity
        users = set(hoursPerGame_training[g])
        max_sim = -1
        for g2, _ in hoursPerUser_training[u]:
            sim = jaccard(users, set(hoursPerGame_training[g2]))
            max_sim = max(sim, max_sim)
        # Compute the performance on validation set
        if(max_sim > sim_th or len(hoursPerGame[g]) > pop_th):
            y_pred.append(1)
        else:
            y_pred.append(0)
        
        if(idx < len(dataset)/2):
            y_act.append(1)
        else:
            y_act.append(0)
            
        idx += 1
    (TP, TN, FP, FN) = compute_confusion_matrix(y_act, y_pred)
    (TPR, TNR, FPR, FNR, BER) = compute_rates(TP, TN, FP, FN)
    acc = (TP + TN) / (FP + FN + TP + TN)
    return (acc, BER)

In [15]:
sim_thres = [i for i in np.arange(0.01, 0.1, 0.01)]
pop_thres = [i for i in np.arange(20,100,20)]
accuracies = defaultdict(dict)
max_acc, max_sim_th, max_pop_th, max_ber = -1, -1, -1, 0.5
for sim_th in sim_thres:
    for pop_th in pop_thres:
        acc, BER = sim_pop(new_validation_set, sim_th, pop_th)
        accuracies[sim_th][pop_th] = acc
        print("Accuracy and BER at sim_th, pop_th = {}, {} are {} and {}".format(sim_th, pop_th, acc, BER))
        if(BER < max_ber):
            max_ber = BER
            max_acc = acc
            max_sim_th = sim_th
            max_pop_th = pop_th
            
print("Max accuracy of {} occurs at sim, pop thresholds = {}, {}".\
      format(max_acc, max_sim_th, max_pop_th))

Accuracy and BER at sim_th, pop_th = 0.01, 20 are 0.5043 and 0.49570000000000003
Accuracy and BER at sim_th, pop_th = 0.01, 40 are 0.5045 and 0.4955
Accuracy and BER at sim_th, pop_th = 0.01, 60 are 0.5042 and 0.49579999999999996
Accuracy and BER at sim_th, pop_th = 0.01, 80 are 0.50415 and 0.49584999999999996
Accuracy and BER at sim_th, pop_th = 0.02, 20 are 0.54045 and 0.45955
Accuracy and BER at sim_th, pop_th = 0.02, 40 are 0.59145 and 0.40854999999999997
Accuracy and BER at sim_th, pop_th = 0.02, 60 are 0.5943 and 0.4057
Accuracy and BER at sim_th, pop_th = 0.02, 80 are 0.5942 and 0.4058
Accuracy and BER at sim_th, pop_th = 0.03, 20 are 0.56885 and 0.43115
Accuracy and BER at sim_th, pop_th = 0.03, 40 are 0.6554 and 0.3446
Accuracy and BER at sim_th, pop_th = 0.03, 60 are 0.6787 and 0.3213
Accuracy and BER at sim_th, pop_th = 0.03, 80 are 0.68515 and 0.31484999999999996
Accuracy and BER at sim_th, pop_th = 0.04, 20 are 0.57505 and 0.42495000000000005
Accuracy and BER at sim_th, po

In [16]:
test_set = []
prediction_file = open("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\predictions_Played_hoursv2.txt", 'w')
for l in open("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\pairs_Played.txt"):
    if l.startswith("userID"):
        #header
        prediction_file.write(l)
        continue
    u,g = l.strip().split('-')
    val = {}
    val['userID'], val['gameID'] = u, g
    test_set.append(val)

# Similarity Popularity of a given dataset
def sim_pop(dataset, sim_th, pop_th):
    idx, numerator = 0, 0
    y_pred = []
    y_act = []
    for d in dataset:
        u, g = d['userID'], d['gameID']
        # Use hoursPerGame for popularity
        # User based comparison for similarity
        users = set(hoursPerGame_training[g])
        max_sim = -1
        for g2, _ in hoursPerUser_training[u]:
            sim = jaccard(users, set(hoursPerGame_training[g2]))
            max_sim = max(sim, max_sim)
        # Compute the performance on validation set
        if(max_sim > sim_th or len(hoursPerGame[g]) > pop_th):
            prediction_file.write(u + "-" + g + ",1\n")
        else:
            prediction_file.write(u + "-" + g + ",0\n")

sim_pop(test_set, 0.04, 80)
print("Completed writing!")
prediction_file.close()

Completed writing!


In [ ]:
def sim_pop_features(dataset):
    features = []
    for d in dataset:
        u, g = d['userID'], d['gameID']
        # Use hoursPerGame for popularity
        # User based comparison for similarity
        users = set(hoursPerGame_training[g])
        max_sim = -1
        for g2, _ in hoursPerUser_training[u]:
            sim = jaccard(users, set(hoursPerGame_training[g2]))
            max_sim = max(sim, max_sim)
        # Compute the performance on validation set
        features.append([1, max_sim, len(hoursPerGame[g])])
    return features

In [53]:
# Perform K Fold cross validation scheme
def kFoldCV(dataset, K):
    random.shuffle(dataset)
    lengthGroup = len(dataset) // K
    idx = 0
    train, test = defaultdict(), defaultdict()
    
    while(idx < K):
        test_set = dataset[idx*lengthGroup : (idx+1) * lengthGroup]
        if(idx == 0):
            train_set = dataset[(idx+1) * lengthGroup:]
        elif(idx > 1 and idx < K - 1):
            train_set = dataset[:idx * lengthGroup] + dataset[(idx+1)*lengthGroup:]
        elif(idx == K-1):
            train_set = dataset[:(idx * lengthGroup)]
        idx += 1
        train[idx] = train_set
        test[idx] = test_set
    
    return train, test

In [ ]:
dataset[0]
print(len(dataset[0]['date']))
print(type(dataset[0]['hours_transformed']))

In [ ]:
# X_train = sim_pop_features(new_training_set)
# Y_train = [1 if i < len(new_training_set)/2 else 0 for i in range(len(new_training_set))]

clf = linear_model.LogisticRegression(C=10**-7, class_weight='balanced', max_iter = 2000)
K = 5
train, test = kFoldCV(new_training_set, K)
# Here, we are building a classifier
for i in range(K):
    train_set = train[i+1]
    test_set = test[i+1]
    print("Lengths of train and test sets are {} and {}".format(len(train_set), len(test_set)))
    X_train = sim_pop_features(train_set)
    Y_train = [1 if (len(d['date']) > 0 or d['hours_transformed'] > 0) \
               else 0 for d in train_set]
    clf.fit(X_train, Y_train)
    print("Done with training for iteration " + str(i))
    X_val = sim_pop_features(test_set + new_validation_set)
    Y_val_act = [1 if (len(d['date']) > 0) or (d['hours_transformed'] > 0) \
               else 0 for d in (test_set + new_validation_set)]

    Y_val_pred = clf.predict(X_val)
    (TP, TN, FP, FN) = compute_confusion_matrix(Y_val_act,Y_val_pred)
    (TPR, TNR, FPR, FNR, BER) = compute_rates(TP, TN, FP, FN)
    acc = (TP + TN) / (TP + TN + FP + FN)
    print("Accuracy, BER at iteration {} are {} and {}".format(i+1, acc, BER))

In [ ]:
X_test = sim_pop_features(new_validation_set)
Y_test = [1 if (len(d['date']) > 0 or (d['hours_transformed'] > 0)) else 0 for d in new_validation_set]
Y_test_pred = clf.predict(X_test)
(TP, TN, FP, FN) = compute_confusion_matrix(Y_test, Y_test_pred)
(TPR, TNR, FPR, FNR, BER) = compute_rates(TP, TN, FP, FN)
acc = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy on the test set is {} and BER is {}".format(acc, BER))

In [66]:
# X_train = sim_pop_features(new_training_set)
# X_val = sim_pop_features(new_validation_set)
# Y_train = [1 if(len(d['date']) > 0) else 0 for d in new_training_set]
# Y_val = [1 if len(d['date']) > 0 else 0 for d in new_validation_set]
#model = sklearn.linear_model.LogisticRegression(C=1.0, )
lambdas = [10 ** i for i in np.arange(-8,-7,0.1, dtype=float)]
for lamb in lambdas:
    model = linear_model.LogisticRegression(C=lamb, class_weight='balanced', max_iter=2000)
    model.fit(X_train, Y_train)
    Y_val_pred = model.predict(X_val)
    #acc = sum(Y_test == validation_predictions)/len(validation_predictions)
    (TP, TN, FP, FN) = compute_confusion_matrix(Y_val, Y_val_pred)
    (TPR, TNR, FPR, FNR, BER) = compute_rates(TP, TN, FP, FN)
    acc = (TP + TN) / (TP + TN + FP + FN)
    print("Accuracy = {} at lambda = {}".format(acc, lamb))
    print("BER = {} at lambda = {}".format(BER, lamb))

Accuracy = 0.6922 at lambda = 1e-08
BER = 0.3078 at lambda = 1e-08
Accuracy = 0.6922 at lambda = 1.2589254117941661e-08
BER = 0.3078 at lambda = 1.2589254117941661e-08
Accuracy = 0.69245 at lambda = 1.584893192461111e-08
BER = 0.30755 at lambda = 1.584893192461111e-08
Accuracy = 0.69245 at lambda = 1.9952623149688747e-08
BER = 0.30755 at lambda = 1.9952623149688747e-08
Accuracy = 0.69245 at lambda = 2.5118864315095718e-08
BER = 0.30755 at lambda = 2.5118864315095718e-08
Accuracy = 0.69265 at lambda = 3.1622776601683666e-08
BER = 0.30735 at lambda = 3.1622776601683666e-08
Accuracy = 0.69265 at lambda = 3.981071705534953e-08
BER = 0.30735 at lambda = 3.981071705534953e-08
Accuracy = 0.69265 at lambda = 5.011872336272694e-08
BER = 0.30735 at lambda = 5.011872336272694e-08
Accuracy = 0.69265 at lambda = 6.309573444801891e-08
BER = 0.30735 at lambda = 6.309573444801891e-08
Accuracy = 0.69265 at lambda = 7.943282347242757e-08
BER = 0.30735 at lambda = 7.943282347242757e-08


In [1]:
# def one_hot_encoding(current_length, max_len):
#     encoding = [0] * (max_len+1)
#     assert(current_length <= max_len)
#     encoding[current_length] = 1
#     return encoding

def sim_pop_features(dataset):
    features = []
    for d in dataset:
        u, g = d['userID'], d['gameID']
        # Use hoursPerGame for popularity
        # User based comparison for similarity
        users = set(hoursPerGame_training[g])
        max_sim = -1
        for g2, _ in hoursPerUser_training[u]:
            sim = jaccard(users, set(hoursPerGame_training[g2]))
            max_sim = max(sim, max_sim)
        # Compute the performance on validation set
        features.append([1, max_sim, len(hoursPerGame[g])])
    return features

In [74]:
X_train = sim_pop_features(new_training_set)
X_val = sim_pop_features(new_validation_set)
Y_train = [1 if(len(d['date']) > 0) else 0 for d in new_training_set]
Y_val = [1 if len(d['date']) > 0 else 0 for d in new_validation_set]
lambdas = [10 ** i for i in np.arange(-10,10, dtype=float)]

for lamb in lambdas:
    model = linear_model.LogisticRegression(C=lamb, class_weight='balanced', max_iter=2000)
    model.fit(X_train, Y_train)
    Y_val_pred = model.predict(X_val)
    #acc = sum(Y_test == validation_predictions)/len(validation_predictions)
    (TP, TN, FP, FN) = compute_confusion_matrix(Y_val, Y_val_pred)
    (TPR, TNR, FPR, FNR, BER) = compute_rates(TP, TN, FP, FN)
    acc = (TP + TN) / (TP + TN + FP + FN)
    print("Accuracy = {} at lambda = {}".format(acc, lamb))
    print("BER = {} at lambda = {}".format(BER, lamb))

Max length = 1092
Max length = 1092


MemoryError: Unable to allocate 2.69 GiB for an array with shape (330000, 1095) and data type float64

In [71]:
model = linear_model.LogisticRegression(C=10**-7, class_weight='balanced', max_iter=2000)
model.fit(X_train, Y_train)
# Test set
test_set = []
prediction_file = open("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\predictions_Played.txt", 'w')
for l in open("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\pairs_Played.txt"):
    if l.startswith("userID"):
        #header
        prediction_file.write(l)
        continue
    u,g = l.strip().split('-')
    val = {}
    val['userID'], val['gameID'] = u, g
    test_set.append(val)
    
X_test = sim_pop_features(test_set)
Y_pred = model.predict(X_test)

for i in range(len(Y_pred)):
    u, g = test_set[i]['userID'], test_set[i]['gameID']
    prediction_file.write(u + "-" + g + "," + str(Y_pred[i]) + "\n")

prediction_file.close()
print("Completed writing the contents!")

Completed writing the contents!
